In [1]:
!pip install sentence-transformers==2.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.4 MB/s eta 0:00:00


In [2]:
import os
import csv
import time

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import http_get

if __name__=='__main__':
    url='http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv'
    dataset_path='quora_duplicate_questions.tsv'
#     max_corpus_size=50000
    
    if not os.path.exists(dataset_path):
        http_get(url, dataset_path)
    
    # get all unique sentences from the file
    corpus_sentences=set()
    with open(dataset_path, encoding='utf8') as fIn:
        reader=csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            corpus_sentences.add(row['question1'])
            corpus_sentences.add(row['question2'])
#             if len(corpus_sentences)>=max_corpus_size:
#                 break
                
    corpus_sentences=list(corpus_sentences)
    model=SentenceTransformer('all-MiniLM-L6-v2').to('cuda')
    model.max_seq_length=256
    
    pool=model.start_multi_process_pool()
    
    # computing the embeddings using the multi-process pool
    emb=model.encode_multi_process(corpus_sentences, pool,batch_size=128,chunk_size=1024,normalize_embeddings=True)
    print('Embeddings computed. Shape:', emb.shape)
    
    # optional : stop the processes in the pool
    model.stop_multi_process_pool(pool)

  0%|          | 0.00/58.2M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings computed. Shape: (537362, 384)


In [3]:
emb.shape

(537362, 384)

In [4]:
import pandas as pd

corpus_embedding=pd.DataFrame(emb)
corpus_embedding.to_csv('quora_duplicate_questions.csv',index=False)